In [ ]:
from plotnine import *

import numpy as np
import pandas as pd

## LMC

In [ ]:
lmc_file = "../data/interim/lmc/RRab.csv"
lmc = pd.read_csv(lmc_file)

AMPLITUDE = "I_amplitude"
PERIOD = "period"

USED_COLUMNS = [PERIOD, AMPLITUDE]

lmc = lmc[USED_COLUMNS]

lmc = lmc.dropna()

print(lmc.info())
lmc.describe()

Let's compute a density estimate for the data points. This will let us see the clustering of the data and help later on for narrowing down the data for curve fitting.

In [ ]:
from scipy import stats

x_a = np.array(lmc[PERIOD])
y_a = np.array(lmc[AMPLITUDE])
points = np.vstack([x_a.ravel(), y_a.ravel()])

xmin, xmax = min(x_a), max(x_a)
ymin, ymax = min(y_a), max(y_a)

x, y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([x.ravel(), y.ravel()])
values = np.vstack([x_a, y_a])
kernel = stats.gaussian_kde(values)

In [ ]:
lmc["density"] = kernel.evaluate(points)

In [ ]:
ggplot(lmc, aes(PERIOD, AMPLITUDE, color="density")) +\
    geom_point() +\
    xlab("Period (days)") +\
    ylab("Amplitude I band (mag)") +\
    ggtitle("OGLE IV LMC - Period-Amplitude Density") +\
    xlim(0.35, 1.0) +\
    ylim(0.0, 1.1)

Here we can see that there is a section of the data plot where the points cluster densly along a curve. The boundry between the Oost I and Oost II groups also looks like it follows a similar trend.

Let's try to get the equation for that line.

If we just were to take the overall trend of the data, then the resulting fit curve would be different from that of the dense area, so let's just fit the curve to the more dense area.

In [ ]:
dense = lmc[lmc["density"] > 12]

In [ ]:
ggplot(dense, aes(PERIOD, AMPLITUDE, color="density")) +\
    geom_point()

For the curve model to fit to the data we will go with a 3rd degree polynomial. A 2nd degree polynomial seems like it would fit the curve better, but using a 3rd degree polynomial will help improve the clustering boundary results.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

X_reg = dense[PERIOD].values.reshape(-1, 1)
y_reg = dense[AMPLITUDE].values.reshape(-1, 1)

poly = PolynomialFeatures(degree=3)
reg = LinearRegression()

poly_regression = make_pipeline(poly, reg)

poly_regression.fit(X_reg, y_reg)

The equation for the line we have fit is the following:

In [ ]:
coef = reg.coef_

print("amp = %f + %f * per + %f * per^2 + %f * per^3" % \
      (coef[0][0] + reg.intercept_, coef[0][1], coef[0][2], coef[0][3]))

Now let's apply the fit curve function to the period values in the dataset so that we can overlay the curve on the data and see how well it fits.

In [ ]:
X_curve = lmc.as_matrix([PERIOD])
y_curve = poly_regression.predict(X_curve)

lmc["curve"] = y_curve

In [ ]:
ggplot(lmc, aes(PERIOD, AMPLITUDE, color="density")) +\
    geom_point() +\
    geom_point(aes(PERIOD, "curve"), color="Red") +\
    ylim(0.0, 1.25)

The curve seems to fit the data reasonably well. The 3rd degree nature of the fit seems odd at the ends of the data space, but it will help in the clustering.

Now that we have the equation for the curve, let's try warping the data space around the curve to create a new feature to use for clustering.

In [ ]:
lmc["y"] = lmc[AMPLITUDE] - lmc["curve"]

In [ ]:
ggplot(lmc, aes(PERIOD, "y", color="density")) +\
    geom_point() +\
    xlab("Period (days)") +\
    ylab("Signal subtracted Amplitude I band (mag)") +\
    ggtitle("OGLE IV LMC - Signal Subtracted Density")

Since the boundary between the Oost I and Oost II seems to follow a similar shape to the curve we bent the space over, in this new feature `y` a horizontal line should form a properly shaped decision boundary.

Let's now try doings some clustering using this new feature. For now we will use KMeans with 3 clusters as this provides good clustering in relation to the Oosterhoff groups.

A similar approach with 2 clusters was attempted, but it did not properly approximate the boundary between the Oosterhoff groups.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

X_clustering = lmc.as_matrix(["y"])

num_clusters = 3
clustering = KMeans(n_clusters=num_clusters, random_state=0)
scaler = StandardScaler()

clustering_pipe = make_pipeline(scaler, clustering)

clusters = clustering_pipe.fit_predict(X_clustering)

In [ ]:
lmc["clusters"] = clusters

In [ ]:
ggplot(lmc, aes(PERIOD, "y", color="clusters")) +\
    geom_point()

In [ ]:
ggplot(lmc.iloc[:5000], aes(PERIOD, AMPLITUDE, color="clusters")) +\
    geom_point()

Here we can see that the points in the cluster `0` appear to be those in the Oost I group, and the points in the clusters `1` and `2` appear to be those in the Oost II group.

Let's simplify these clusters into a binary feature for Oosterhoff group.

In [ ]:
LMC_OOST_1_CLUSTER = 1

lmc["is_oost_ii"] = lmc["clusters"].map(lambda x: x != LMC_OOST_1_CLUSTER)

In [ ]:
ggplot(lmc, aes(PERIOD, AMPLITUDE, color="is_oost_ii")) +\
    geom_point() +\
    ylim(0.0, 1.25) +\
    xlab("Period (days)") +\
    ylab("Amplitude I band (mag)") +\
    ggtitle("OGLE IV LMC - Oosterhoff Groups")

In [ ]:
ggplot(lmc, aes(PERIOD, AMPLITUDE, color="density")) +\
    facet_wrap("~is_oost_ii") +\
    geom_point()

So now that we have performed the clustering, let's get the equation for the decision boundary between the two Oosterhoff groups.

In [ ]:
boundary_y = lmc[lmc["clusters"] == LMC_OOST_1_CLUSTER]["y"].max()

In [ ]:
print("amp = %f + %f * per + %f * per^2 + %f * per^3" % \
      (boundary_y + coef[0][0] + reg.intercept_, coef[0][1], coef[0][2], coef[0][3]))

Let's plot the decision boundary to double check that we have the correct equation.

In [ ]:
X_boundary = lmc[PERIOD].values.reshape(-1, 1)
y_boundary = poly_regression.predict(X_boundary) + boundary_y

lmc["boundary"] = y_boundary

In [ ]:
ggplot(lmc, aes(PERIOD, AMPLITUDE, color="density")) +\
    geom_point() +\
    geom_point(aes(PERIOD, "boundary"), color="Red") +\
    ylim(0.0, 1.25) +\
    xlab("Period (days)") +\
    ylab("Amplitude I band (mag)")

## SMC
Now let's try applying the same approach to the SMC data.

In [ ]:
smc_file = "../data/interim/smc/RRab.csv"
smc = pd.read_csv(smc_file)

smc = smc[USED_COLUMNS]

smc = smc.dropna()

print(smc.info())
smc.describe()

In [ ]:
from scipy import stats

x_a = np.array(smc[PERIOD])
y_a = np.array(smc[AMPLITUDE])
points = np.vstack([x_a.ravel(), y_a.ravel()])

xmin, xmax = min(x_a), max(x_a)
ymin, ymax = min(y_a), max(y_a)

x, y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([x.ravel(), y.ravel()])
values = np.vstack([x_a, y_a])
kernel = stats.gaussian_kde(values)

In [ ]:
smc["density"] = kernel.evaluate(points)

In [ ]:
ggplot(smc, aes(PERIOD, AMPLITUDE, color="density")) +\
    geom_point() +\
    xlab("Period (days)") +\
    ylab("Amplitude I band (mag)") +\
    ggtitle("OGLE IV SMC - Period-Amplitude Density") +\
    xlim(0.35, 1.0) +\
    ylim(0.0, 1.1)

In [ ]:
dense_smc = smc[smc["density"] > 17] # 23

In [ ]:
ggplot(dense_smc, aes(PERIOD, AMPLITUDE, color="density")) +\
    geom_point()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

X_reg = dense_smc[PERIOD].values.reshape(-1, 1)
y_reg = dense_smc[AMPLITUDE].values.reshape(-1, 1)

poly = PolynomialFeatures(degree=3)
reg = LinearRegression()

poly_regression = make_pipeline(poly, reg)

poly_regression.fit(X_reg, y_reg)

In [ ]:
coef = reg.coef_

print("amp = %f + %f * per + %f * per^2 + %f * per^3" % \
      (coef[0][0] + reg.intercept_, coef[0][1], coef[0][2], coef[0][3]))

In [ ]:
X_curve = smc[PERIOD].values.reshape(-1,1)
y_curve = poly_regression.predict(X_curve)

smc["curve"] = y_curve

In [ ]:
ggplot(smc, aes(PERIOD, AMPLITUDE, color="density")) +\
    geom_point() +\
    geom_point(aes(PERIOD, "curve"), color="Red") +\
    ylim(0.0, 1.25)

In [ ]:
smc["y"] = smc[AMPLITUDE] - smc["curve"]

In [ ]:
ggplot(smc, aes(PERIOD, "y", color="density")) +\
    geom_point()

In [ ]:
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

X_clustering = smc["y"].values.reshape(-1, 1)

num_clusters = 3
clustering = KMeans(n_clusters=num_clusters, random_state=0)
scaler = StandardScaler()

clustering_pipe = make_pipeline(scaler, clustering)

clusters = clustering_pipe.fit_predict(X_clustering)

In [ ]:
smc["clusters"] = clusters

In [ ]:
ggplot(smc, aes(PERIOD, "y", color="clusters")) +\
    geom_point()

In [ ]:
ggplot(smc, aes(PERIOD, AMPLITUDE, color="clusters")) +\
    geom_point()

In [ ]:
SMC_OOII_CLUSTER = 0

smc["is_oost_ii"] = smc["clusters"].map(lambda x: x != SMC_OOII_CLUSTER)

In [ ]:
ggplot(smc, aes(PERIOD, AMPLITUDE, color="is_oost_ii")) +\
    geom_point() +\
    xlab("Period (days)") +\
    ylab("Amplitude I band (mag)") +\
    ggtitle("OGLE IV SMC - Oosterhoff Groups")

In [ ]:
ggplot(smc, aes(PERIOD, AMPLITUDE, color="density")) +\
    facet_wrap("~is_oost_ii") +\
    geom_point()

In [ ]:
boundary_y = smc[smc["clusters"] == SMC_OOII_CLUSTER]["y"].max()

In [ ]:
print("amp = %f + %f * per + %f * per^2 + %f * per^3" % \
      (boundary_y + coef[0][0] + reg.intercept_, coef[0][1], coef[0][2], coef[0][3]))

In [ ]:
X_boundary = smc[PERIOD].values.reshape(-1, 1)
y_boundary = poly_regression.predict(X_boundary) + boundary_y

smc["boundary"] = y_boundary

In [ ]:
ggplot(smc, aes(PERIOD, AMPLITUDE, color="density")) +\
    geom_point() +\
    geom_point(aes(PERIOD, "boundary"), color="red") +\
    ylim(0.0, 1.25)